In [11]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.impute import KNNImputer, SimpleImputer


data_missing_less_40p = pd.read_csv('Assignment_1_data_missing_less_40p.csv', index_col=False)
noncontinuous_encoded = pd.read_csv('noncontinuous_encoded.csv', index_col=False)

less_40p_encoded = pd.concat([noncontinuous_encoded['gender'], data_missing_less_40p.drop('gender',axis= 1)], axis= 1)
less_40p_encoded.to_csv('less_40p_encoded.csv', index=False)  #comment to not write data


In [12]:
def make_knn_imputer(table, name_select_first_col, name_select_remaining_col):
    knn_imputer = KNNImputer()
    
    X = table[name_select_first_col].to_numpy()
    X = X.reshape(-1, 1)    
   
    knn_imputed_init = pd.DataFrame(knn_imputer.fit_transform(X), columns = [name_select_first_col])
    
    for i in name_select_remaining_col:
        X = lambda i: table[i].to_numpy()
        X = X(i).reshape(-1, 1)
        
        knn_imputed_temp = pd.DataFrame(knn_imputer.fit_transform(X), columns = [i])
        knn_imputed_init = pd.concat([knn_imputed_init, knn_imputed_temp], axis=1)
    
    output = knn_imputed_init
    
    return output

knn_imputed_less_40 = make_knn_imputer(less_40p_encoded, 'gender', list(less_40p_encoded)[1:])
knn_imputed_less_40.to_csv('knn_imputed_less_40.csv', index=False)  #comment to not write data

#def addFeatures(feature_table, target_table):
#    return pd.concat([target_table, feature_table], axis=1)

def make_imp_median(table, name_select_first_col, name_select_remaining_col):
    imp_median = SimpleImputer(strategy='median')
    
    X = table[name_select_first_col].to_numpy()
    X = X.reshape(-1, 1)    
   
    imp_median_init = pd.DataFrame(imp_median.fit_transform(X), columns = [name_select_first_col])
    
    for i in name_select_remaining_col:
        X = lambda i: table[i].to_numpy()
        X = X(i).reshape(-1, 1)
        
        imp_median_temp = pd.DataFrame(imp_median.fit_transform(X), columns = [i])
        imp_median_init = pd.concat([imp_median_init, imp_median_temp], axis=1)
    
    output = imp_median_init
    
    return output

median_imputed_less_40 = make_imp_median(less_40p_encoded, 'gender', list(less_40p_encoded)[1:])
median_imputed_less_40.to_csv('median_imputed_less_40.csv', index=False) #comment to not write data


smoten on the imputed

In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTEN

median_imputed_less_40 = pd.read_csv('median_imputed_less_40.csv', index_col=False)

pd.set_option("display.max_columns", None)

def smotenThis(table, lst, y_var):    
    X = table[lst]
    y = table[y_var].to_numpy()
    y = y.ravel()
    y = LabelEncoder().fit_transform(y)

    oversample = SMOTEN()
    X, y = oversample.fit_resample(X, y)    
    array_X = pd.DataFrame(X)
    array_y = pd.DataFrame(y, columns = [y_var])
    
    output = pd.concat([array_X, array_y], axis=1)
    
    return output

smoten_median_imputed_less_40 = smotenThis(median_imputed_less_40, list(median_imputed_less_40.drop('outcome',axis= 1)), 'outcome')
smoten_median_imputed_less_40.to_csv('smoten_median_imputed_less_40.csv', index=False)  #comment to not write data


    
